NOVA VERSÂO AGORA COM InceptionV3Model

In [3]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

TRAINING_DIR = "./data/train"
VALIDATION_DIR = "./data/valid"

training_datagen = ImageDataGenerator(rescale=1./255,zoom_range=0.15,horizontal_flip=True,fill_mode='nearest')
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = training_datagen.flow_from_directory(TRAINING_DIR, target_size=(224,224), shuffle=True, class_mode='categorical', batch_size=128)
validator_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, target_size=(224,224), shuffle=True, class_mode='categorical', batch_size=14)

Found 6166 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


In [4]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout

input_tensor = Input(shape=(224,224,3))
base_model = InceptionV3(input_tensor = input_tensor, weights='imagenet', include_top = False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)

predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

history = model.fit(
    train_generator,
    steps_per_epoch = 14,
    epochs = 20,
    validation_data=validator_generator,
    validation_steps = 14
)

Epoch 1/20
12/14 [========================>.....] - ETA: 11s - loss: 4.6616 - acc: 0.7133

C:\Users\Matheus\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


14/14 [==============================] - 84s 6s/step - loss: 4.0753 - acc: 0.7195
Epoch 2/20
14/14 [==============================] - 81s 6s/step - loss: 0.4564 - acc: 0.8304
Epoch 3/20
14/14 [==============================] - 83s 6s/step - loss: 0.3709 - acc: 0.8772
Epoch 4/20
14/14 [==============================] - 77s 5s/step - loss: 0.3847 - acc: 0.8778
Epoch 5/20
14/14 [==============================] - 73s 5s/step - loss: 0.3572 - acc: 0.8655
Epoch 6/20
14/14 [==============================] - 69s 5s/step - loss: 0.2972 - acc: 0.9069
Epoch 7/20
14/14 [==============================] - 79s 6s/step - loss: 0.2774 - acc: 0.9122
Epoch 8/20
14/14 [==============================] - 90s 6s/step - loss: 0.3634 - acc: 0.8700
Epoch 9/20
14/14 [==============================] - 88s 6s/step - loss: 0.2598 - acc: 0.9051
Epoch 10/20
14/14 [==============================] - 89s 6s/step - loss: 0.2331 - acc: 0.9163
Epoch 11/20
14/14 [==============================] - 91s 6s/step - loss: 0.2414 

In [5]:
for layer in model.layers[:249]:
    layer.trainable = False

for layer in model.layers[249:]:
    layer.trainable = True

from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),loss='categorical_crossentropy',metrics=['acc'])

history = model.fit(
    train_generator,
    steps_per_epoch = 14,
    epochs =10,
    validation_data = validator_generator,
    validation_steps = 14
)

C:\Users\Matheus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/10
14/14 [==============================] - 116s 7s/step - loss: 0.3102 - acc: 0.9247
Epoch 2/10
14/14 [==============================] - 103s 7s/step - loss: 0.2903 - acc: 0.9217
Epoch 3/10
14/14 [==============================] - 110s 8s/step - loss: 0.2645 - acc: 0.9408
Epoch 4/10
14/14 [==============================] - 111s 8s/step - loss: 0.2562 - acc: 0.9386
Epoch 5/10
14/14 [==============================] - 111s 8s/step - loss: 0.2453 - acc: 0.9353
Epoch 6/10
14/14 [==============================] - 102s 7s/step - loss: 0.2466 - acc: 0.9253
Epoch 7/10
14/14 [==============================] - 104s 7s/step - loss: 0.2310 - acc: 0.9306
Epoch 8/10
14/14 [==============================] - 77s 5s/step - loss: 0.2248 - acc: 0.9353
Epoch 9/10
14/14 [==============================] - 79s 6s/step - loss: 0.1973 - acc: 0.9425
Epoch 10/10
14/14 [==============================] - 74s 6s/step - loss: 0.1947 - acc: 0.9407


In [6]:
import joblib

model.save('modeloV3.h5')
joblib.dump(history, "modeloV3.pkl")

INFO:tensorflow:Assets written to: ram://8dca158c-6e8a-49d8-bcb2-20d62714854d/assets


['modeloV3.pkl']

In [7]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

saved_model = load_model('modeloV3.h5')
saved_history = joblib.load("modeloV3.pkl")

plt.plot(saved_model.history['accuracy'])
plt.plot(saved_model.history['val_accuracy'])
plt.title('Acuracia modelo v3')
plt.xlabel('épocas')
plt.ylabel('acurácias')
plt.legend(['treino','avaliação'])

FileNotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for ram://d64a916c-c68f-4089-8d7e-9128c97d7bcd/variables/variables
 You may be trying to load on a different device from the computational device. Consider setting the `experimental_io_device` option in `tf.saved_model.LoadOptions` to the io_device such as '/job:localhost'.